# Lab 2: Working with Anthropic API and Reasoning Models

### Overview

Let's finally dive into the nitty-gritty and see how we can code with Claude.

This notebook explores the Anthropic API, focusing on how to leverage both standard chat capabilities and advanced reasoning features. We'll examine how to structure effective system prompts and create applications that utilize the model's extended thinking capabilities.

### Objectives
- Set up and configure the Anthropic API
- Create tailored system prompts for specific use cases
- Compare standard responses with reasoning-enhanced outputs
- Analyze the "thinking" process in reasoning models

#### Step 1: Import libraries and load the environment variables

In [3]:
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

client = anthropic.Client()

#### Step 2: Call the API

In [ ]:


messages = client.messages.create(
    model="claude-3-5-haiku-latest",
    system="You are an unhepful assistant.",
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ],
    max_tokens=100
)

# print(messages)
print(messages.content[0].text)


Paris, but I'm being deliberately unhelpful about it. *shrugs*


#### Step 3: Customize the Prompt

Customer service representative

In [4]:
complaint = "You have over charged me for this product. I demand a refund."
company_policy = """
- We offer refunds only within 14 days of purchase.
- For purchases over $100, a supervisor must approve the refund.
- We prioritize customer satisfaction but aim to minimize refund abuse.

"""

In [8]:
system_message = f"""You are a refund request assistant and need to impose company policy {company_policy} when responding to customer complaints."""

prompt = f"""
A customer has made the following complaint: {complaint}.
Keep your response concise and to the point, while adhering to the company policy.
"""
messages2 = client.messages.create(
    model="claude-3-5-haiku-latest",
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ],
    max_tokens=200
)   
print(messages2.content[0].text)

chat_message = messages2.content[0].text
chat_message 

I understand your concern. To process a refund, could you please provide:
1. Your order number
2. Date of purchase
3. Specific details about the overcharge

Our policy allows refunds within 14 days of purchase, and I'll be happy to help you resolve this.


"I understand your concern. To process a refund, could you please provide:\n1. Your order number\n2. Date of purchase\n3. Specific details about the overcharge\n\nOur policy allows refunds within 14 days of purchase, and I'll be happy to help you resolve this."

#### Step 4: Rate the response using a reasoning model

In [9]:
system_message = "You are a senior customer service agent with advanced reasoning capabilities. You are to analyze customer complaints and provide a detailed response based on company policy."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_message}

Company Policy:
{company_policy}

Based on this, decide how the support team should respond. Your analysis must include:
- Whether the response is appropriate given the complaint and company policy
- Professional tone and empathy

"""

reasoning_message = client.messages.create(
    model="claude-3-7-sonnet-latest",
    thinking={
        'type': 'enabled',
        'budget_tokens': 2000
    },
    max_tokens=3000,
    temperature=1,
    system=system_message,
    messages=[
        {"role": "user", "content": reasoning_prompt}
    ]
)

reasoning_message = reasoning_message.content

print(reasoning_message)


[ThinkingBlock(signature='ErUBCkYIBxgCIkAV0Ob4QQEZY1XEpiDWwL2h/8Bmi1NCES/EactF6oCK2y3b9DDn9JBlcpwMdZRJmjNtcfQxDWhADxJ/zpuUxxxSEgxCQV+S0CHI+YU+XKMaDIh8trY5zlt4LMeo9SIwZH2jb/UI3roDas8fkoYhVvcpR38nHeN6u4tswncLedYC381pMGAVHfuI12gjIyhGKh3XiguIl0IkpZ76xUUmbGrkmX/mTMJDCTuKdMmd1RgC', thinking='Let\'s evaluate the customer service email response based on the information provided.\n\nFirst, let\'s review what we have:\n\n1. The customer complaint is brief and states: "You have over charged me for this product. I demand a refund."\n\n2. The response from the customer service agent is:\n"I understand your concern. To process a refund, could you please provide:\n1. Your order number\n2. Date of purchase\n3. Specific details about the overcharge\n\nOur policy allows refunds within 14 days of purchase, and I\'ll be happy to help you resolve this."\n\n3. Company Policy:\n- Refunds only within 14 days of purchase.\n- For purchases over $100, a supervisor must approve the refund.\n- Prioritize customer 

In [16]:
#reasoning_message consists of two elements which are objects, so we use . to access the text
reasoning_message = reasoning_message[1].text
# len(reasoning_message)

In [17]:
# reasoning_message = reasoning_message[1].text

display(Markdown(reasoning_message))

# Evaluation of Customer Service Response

## Appropriateness of Response

The customer service response is **appropriate and well-structured** given both the complaint and company policy for these reasons:

- **Information gathering**: The agent correctly requests essential information (order number, purchase date, overcharge details) needed to verify the claim before proceeding.
- **Policy transparency**: The 14-day refund window is clearly communicated upfront, preventing potential disappointment later.
- **Solution-focused approach**: Rather than becoming defensive about the overcharge accusation, the agent moves directly to resolution steps.

## Professional Tone and Empathy

The response demonstrates strong professional communication:

- **Acknowledgment**: "I understand your concern" validates the customer's frustration without admitting fault or making promises.
- **Helpful attitude**: The agent expresses willingness to assist, positioning themselves as an ally rather than an obstacle.
- **Balanced approach**: The tone strikes an appropriate balance between empathy and procedural requirements.
- **De-escalation**: The response naturally de-escalates the demanding tone of the complaint by focusing on concrete next steps.

## Areas for Consideration

While the response is generally excellent, the agent might consider:

- Mentioning the supervisor approval requirement for purchases over $100 if the agent can see that the order exceeds this amount
- Including a brief apology for the inconvenience without admitting fault

Overall, this is an effective response that adheres to company policy while maintaining a customer-centric approach.

# **Your Challenge!!!**

A customer has submitted a complaint. Based on company policy and the situation, the model must select the correct resolution option from a list, and give a 1-line justification.

In [18]:
complaint = "I ordered a ceramic mug and received a broken one. Please fix this!"

policy = """
- Broken or damaged items must be reported within 7 days of delivery.
- If reported in time, the item is replaced or refunded.
- No action is taken for reports after 7 days unless escalated.
"""

metadata = """
    Reported Days After Delivery: 5
    Item Value: $12
    Customer Tier: Standard
    Past Complaints: 0
"""


Build a customer support agent that could help solve this issue

The agent should choose one of the following options:
- Refund Item
- Replace Item
- Offer Discount
- Apologize (no action)
- Report to supervisor

In [19]:
system_message = "You are a customer support agent."

prompt = f"""You need to choose one of the following 5 actions based on the complaint {complaint}, company policy {policy}, and metadata {metadata}: 
1- Refund Item
2- Replace Item
3- Offer Discount
4- Apologize (no action)
5- Report to supervisor 
rovide a brief explanation for your choice."""

message = client.messages.create(
    model="claude-3-7-sonnet-latest",
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ],
    max_tokens=500
)
print(message.content[0].text)  

Based on the complaint and metadata, I'll choose option 2: Replace Item.

Explanation: The customer reported a broken ceramic mug within 5 days of delivery, which is within the 7-day reporting window specified by company policy. Since the damage was reported in time, the policy clearly states that the item should be replaced or refunded. A replacement is the most appropriate action since it directly addresses the customer's request to "fix this" and provides them with the product they originally ordered in proper condition.


## **Reflections**

1. What reasoning steps did the model take to select its resolution? Were they logical and aligned with company policy? (Hint: Have a look at the reasoning output)
  It considered the policy before making a decision.
2. If you were designing the agent, what would you change about its decision-making process?
  Nothing.
3. What did you learn about prompt design and reasoning models from this exercise?
  Specify the role of the agent. Give as much context as possible for the prompt.